In [2]:
import ipynb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import pickle
from scipy import signal
# import neurokit2 as nk
import random
# %matplotlib inline 
import pyhrv
import ipynb
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import svm

In [3]:
#PCA
def pca_feat(features):
    scaler = StandardScaler().fit(features)
    X_train = scaler.transform(features)

    pca = PCA(n_components=25)
    pca.fit(X_train)
    X_train_pca = pca.transform(X_train)
    x_train = X_train_pca[:,0:3] # Three best features
    return x_train

In [5]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import MinMaxScaler

# Variance Threshold
def var_thres(features):
    scaler = MinMaxScaler().fit(total_features)
    X_train = scaler.transform(total_features)

    X = X_train[0:,0:49]

    v_threshold = VarianceThreshold(threshold=0.005) # Set a threshold
    v_threshold.fit(X)
    index = v_threshold.get_support()
    true_index = [i for i, x in enumerate(index) if x]
    vt_features = [total_features_index[i] for i in true_index]
    return vt_features


In [6]:
def cor_selector(X, y,num_feats):
    cor_list = []
    feature_name = X.columns.tolist()
    # calculate the correlation with y for each feature
    for i in X.columns.tolist():
        cor = np.corrcoef(X[i], y)[0, 1]
        cor_list.append(cor)
    # replace NaN with 0
    cor_list = [0 if np.isnan(i) else i for i in cor_list]
    # feature name
    cor_feature = X.iloc[:,np.argsort(np.abs(cor_list))[-num_feats:]].columns.tolist()
    # feature selection? 0 for not select, 1 for select
    cor_support = [True if i in cor_feature else False for i in feature_name]
    return cor_support, cor_feature

def pears(X):
    cor_support, cor_feature = cor_selector(X, y,6)
    return cor_feature

In [ ]:
scaler = StandardScaler().fit(features)
X_train = scaler.transform(features)

pca = PCA(n_components=25)
pca.fit(X_train)
X_train_pca = pca.transform(X_train)
x_train = X_train_pca[:,0:3] # Three best features

plt.figure(figsize=(8,6))
plt.scatter(x_train[:,0], x_train[:,1], c=y)

In [ ]:
# perform pca on features
plt.bar(range(0,25), pca.explained_variance_ratio_, label="individual var");
plt.step(range(0,25), np.cumsum(pca.explained_variance_ratio_),'r', label="cumulative var");
plt.xlabel('Principal component index'); plt.ylabel('explained variance ratio %');
plt.legend()

In [ ]:
# Information gain
from sklearn.feature_selection import mutual_info_classif
%matplotlib inline

scaler = StandardScaler().fit(features)
X_train = scaler.transform(features)

importances = mutual_info_classif(X_train, y)
feat_importances = pd.Series(importances, features.columns[0:len(features.columns)])
feat_importances.plot(kind='bar', color='teal')
plt.show()

In [ ]:
total_features_index = []

for a in total_features:
    total_features_index.append(a)

print(total_features_index)
print(len(total_features_index))


In [ ]:
# Variance Threshold

from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler().fit(total_features)
X_train = scaler.transform(total_features)

X = X_train[0:,0:49]

v_threshold = VarianceThreshold(threshold=0.005) # Set a threshold
v_threshold.fit(X)
print(v_threshold.variances_)
index = v_threshold.get_support()
print(index)
true_index = [i for i, x in enumerate(index) if x]

# for i in true_index:
#     print(total_features_index[i])
#     print(total_features[total_features_index[i]])
vt_features = [total_features_index[i] for i in true_index]
len(vt_features)
# features = total_features[vt_features]
# features.head()


In [ ]:
#Pearson Correlation
import seaborn as sns

# total_features = pd.merge(edafeatures, ecgfeatures, left_index=True, right_index=True)
out_features = total_features
out_features['y'] = y

# plt.figure(figsize=(12,10))
cor = out_features.corr()
# print(cor)
# sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
# plt.show()
#Correlation with output variable
cor_target = abs(cor["y"])

# print(cor_target)
#Selecting highly correlated features
relevant_features = cor_target[cor_target>0.3]
# features = out_features[relevant_features.index[:-1]]
relevant_features

In [ ]:
# HR_mean, scr_mobility scr_integral 
print(out_features[['HR_mean',"scr_mobility"]].corr())
print(out_features[["HR_mean","scr_integral"]].corr())
print(out_features[["scr_mobility","scr_integral"]].corr())
# features = features[['HR_mean',"scr_mobility","scr_integral"]]
features = out_features[["HR_mean", "scr_mobility", "scr_integral"]]

In [ ]:
X = features
def cor_selector(X, y,num_feats):
    cor_list = []
    feature_name = X.columns.tolist()
    # calculate the correlation with y for each feature
    for i in X.columns.tolist():
        cor = np.corrcoef(X[i], y)[0, 1]
        cor_list.append(cor)
    # replace NaN with 0
    cor_list = [0 if np.isnan(i) else i for i in cor_list]
    # feature name
    cor_feature = X.iloc[:,np.argsort(np.abs(cor_list))[-num_feats:]].columns.tolist()
    # feature selection? 0 for not select, 1 for select
    cor_support = [True if i in cor_feature else False for i in feature_name]
    return cor_support, cor_feature
cor_support, cor_feature = cor_selector(X, y,6)
print(str((cor_feature)), 'selected features')

In [ ]:
#Chi-squared

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler

X=features
X_norm = MinMaxScaler().fit_transform(X)

chi_selector = SelectKBest(chi2, k=6)
chi_selector.fit(X_norm, y)
chi_support = chi_selector.get_support()
chi_feature = X.loc[:,chi_support].columns.tolist()
print(str(chi_feature), 'selected features')

In [ ]:
# # Forward wrapper

# from mlxtend.feature_selection import SequentialFeatureSelector
# ffs = SequentialFeatureSelector(total_features, k_features='best', forward = 'True', n_jobs=-1)
# ffs.fit(X, y)
# features= list(ffs.k_feature_names_)
# features= list(map(int, features))
# lr.fit[x_train[features], y_train]
# y_pred = lr.predict(x_train[features])

In [ ]:
# Backwards
import statsmodels.api as sm

cols = list(total_features.columns)
pmax = 1

while (len(cols)>0):
    p= []
    X_1 = total_features[cols]
    X_1 = sm.add_constant(X_1)
    model = sm.OLS(y,X_1).fit()
    p = pd.Series(model.pvalues.values[1:],index = cols)      
    pmax = max(p)
    feature_with_p_max = p.idxmax()
    if(pmax>0.05):
        cols.remove(feature_with_p_max)
    else:
        break
selected_features_BE = cols
print(selected_features_BE)
features = total_features[selected_features_BE]

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
X = features

In [ ]:
#Wrapper Forwards
sfs = SFS(svm.SVC(),
          k_features=6,
          forward=True,
          floating=False,
          scoring = 'r2',
          cv = 0)
sfs.fit(X, y)
fwrapper_feature = list(sfs.k_feature_names_)
fwrapper_feature

In [ ]:
#Wrapper Backwards
sbs = SFS(svm.SVC(),
         k_features=6,
         forward=False,
         floating=False,
         cv=0)
sbs.fit(X, y)
bwrapper_feature = list(sbs.k_feature_names_)
bwrapper_feature

In [ ]:
#Step-wise wrapper
sffs = SFS(svm.SVC(),
         k_features=(3,11),
         forward=True,
         floating=True,
         cv=0)
sffs.fit(X, y)
swrapper_feature = list(sffs.k_feature_names_)
swrapper_feature

In [ ]:
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs

fig1 = plot_sfs(sfs.get_metric_dict(), kind='std_dev')
plt.title('Sequential Forward Selection (w. StdErr)')
plt.grid()
plt.show()

In [ ]:
features = total_features[feature_namess]

features.head()

In [ ]:
# Lasso
from sklearn.linear_model import LassoCV
from sklearn.model_selection import KFold

X = features
alphas = np.logspace(-4, -0.5, 5)

out_k_fold = KFold(3, shuffle=False)
in_k_fold = KFold(3, shuffle=False)
for (calib, test) in out_k_fold.split(X, y):
    X_calib, y_calib = X[calib[0]:calib[-1]], y[calib[0]:calib[-1]]
    X_test, y_test = X[test[0]:test[-1]], y[test[0]:test[-1]]
    lasso_cv = LassoCV(alphas=alphas, random_state=0, max_iter=10000, cv=in_k_fold)
    lasso_cv.fit(X_calib, y_calib)
    print(lasso_cv.alpha_, lasso_cv.score(X_test, y_test))
    break

In [ ]:
from sklearn.feature_selection import SelectFromModel

sel_ = SelectFromModel(
    LogisticRegression(C=0.5, penalty='l1', solver='liblinear', random_state=10))
sel_.fit(scaler.transform(X_train), y_train)

a =(sel_.estimator_.coef_ == 0).ravel().tolist()

# removed_feats = X_train.columns[(sel_.estimator_.coef_ == 0).ravel().tolist()]
X_train_selected = sel_.transform(scaler.transform(X_train))
X_test_selected = sel_.transform(scaler.transform(X_test))
# selected_columns_lasso = logistic.columns[selected_featuress.var() != 0]

# Y_test_pred = logistic_model.predict(X_test)
# log_score = logistic_model.score(X_test,y_test)

In [ ]:
# LASSO
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression

num_feats = 10
scaler = StandardScaler().fit(features)
X_train = scaler.transform(features)
embeded_lr_selector = SelectFromModel(LogisticRegression(C=1, penalty="l1", solver='liblinear'), max_features=num_feats)
embeded_lr_selector.fit(scaler.transform(X_train), y)

embeded_lr_support = embeded_lr_selector.get_support()
embeded_lr_feature = X.loc[:,embeded_lr_support].columns.tolist()
print(str((embeded_lr_feature)), 'selected features')

In [ ]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier

scaler = StandardScaler().fit(total_features)
X_train = scaler.transform(total_features)

embeded_rf_selector = SelectFromModel(RandomForestClassifier(n_estimators=100), max_features=10)
embeded_rf_selector.fit(X_train, y)

embeded_rf_support = embeded_rf_selector.get_support()
embeded_rf_feature = X.loc[:,embeded_rf_support].columns.tolist()
print(str((embeded_rf_feature)), 'selected features')

In [ ]:
from sklearn.feature_selection import SelectFromModel
from lightgbm import LGBMClassifier

lgbc=LGBMClassifier(n_estimators=500, learning_rate=0.05, num_leaves=32, colsample_bytree=0.2,
            reg_alpha=3, reg_lambda=1, min_split_gain=0.01, min_child_weight=40)

embeded_lgb_selector = SelectFromModel(lgbc, max_features=num_feats)
embeded_lgb_selector.fit(X, y)

embeded_lgb_support = embeded_lgb_selector.get_support()
embeded_lgb_feature = X.loc[:,embeded_lgb_support].columns.tolist()
print(str(len(embeded_lgb_feature)), 'selected features')

In [ ]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier

scaler = StandardScaler().fit(total_features)
X_train = scaler.transform(total_features)

embeded_rf_selector = SelectFromModel(RandomForestClassifier(n_estimators=100), max_features=10)
embeded_rf_selector.fit(X_train, y)

embeded_rf_support = embeded_rf_selector.get_support()
embeded_rf_feature = X.loc[:,embeded_rf_support].columns.tolist()
print(str((embeded_rf_feature)), 'selected features')

In [ ]:
x_train = total_features[embeded_rf_feature]
x_train.head()